In [62]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform

def recommendation_val(lon, lat):
    return np.random.randint(0, 255)

# ************** copied from stack exchange*****************
# https://gis.stackexchange.com/questions/129847/obtain-coordinates-and-corresponding-pixel-values-from-geotiff-using-python-gdal
fname = 'TAXOUSDA_1km_Ethiopia.tiff'
with rasterio.open(fname) as r:
    profile = r.profile
    T0 = r.transform
    p1 = Proj(r.crs)
    A = r.read() 
    

cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))
T1 = T0 * Affine.translation(0.5, 0.5)
rc2en = lambda r, c: (c, r) * T1
eastings, northings = np.vectorize(rc2en, otypes=[np.float, np.float])(rows, cols)

p2 = Proj(proj='latlong',datum='WGS84')

longs, lats = transform(p1, p2, eastings, northings)
# **********************************************************

# reshape
A = A.reshape(A.shape[1], A.shape[2])
# zeroing out numpy array A
A = A & ~A


for i, (lon, lat) in enumerate(zip(longs, lats)):
    for j in range(longs.shape[1]):
        # get recommendation for a specific lon and lat
        # recommendation val function needs to be written
        # Updating array A
        A[i][j] = recommendation_val(lon[j], lat[j])

#Writing it as a tiff
profile.update(
        dtype=rasterio.uint8)

with rasterio.open('recommendation.tif', 'w', **profile) as dst:
    dst.write(A.astype(rasterio.uint8), 1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
